# Tiền xử lý dữ liệu

Mục tiêu: chúng ta load được các dữ liệu từ các file JSON, sau đó sẽ loại bỏ:
1. Cleaning: Loại bỏ các dấu câu đã được chuyển thành NCR (numeric character reference)
2. Tokenizer -> sau đó loại bỏ Stopwords
3. \[Optional\] Chuyển từ viết tắt thành từ đầy đủ


In [1]:
import os,json
import pandas as pd

import unicodedata as ud
import sys

import html

_json&#95;files_: List chứa các file JSON trong data


In [2]:
path_to_news = "Data/"
json_files = [pos_json for pos_json in os.listdir(path_to_news) if pos_json.endswith('.json')]

In [3]:
print(json_files)

['real_1.json', 'fake_1.json']


Tạo dataframe bằng [pandas.DataFrame](http://pandas-docs.github.io/pandas-docs-travis/generated/pandas.Series.loc.html) để có thể lưu các thuộc tính load từ các file JSON đã được load ở trên

In [4]:
news = pd.DataFrame(columns=['title', 'text', 'authors'])

Load dữ liệu từ các file JSON vào list news theo các thuộc tính: _title_, _text_ và _authors_

In [5]:
for index, js in enumerate(json_files):
    with open(os.path.join(path_to_news, js), 'r', encoding='utf-8') as json_file:
        json_text = json.load(json_file)
        
        
        title = json_text['title']
        text = json_text['text']
        author = json_text['authors']
        
        news.loc[index] = [title, text, author]

## 1. Loại bỏ dấu câu

Trong dữ liệu sau khi được crawl về, các ký tự đặc biệt (ví dụ các dấu câu) sẽ được đổi thành dạng [numeric character reference (ncr)](https://en.wikipedia.org/wiki/Numeric_character_reference). Ví dụ như: Trong file _real&#95;1.json_ phần title (ở bên dưới). 

Chúng ta phải đổi lại dưới dạng Unicode để có thể loại bỏ phần dấu câu. 

_html.unescape(str)_: đổi các kí tự dạng __ncr__ về định dạng Unicode của chúng

Tham khảo [stackoverflow](https://stackoverflow.com/a/17018874/5144980)

In [6]:
title_str = news.loc[0]['title']

title_str = html.unescape(title_str)

_remove&#95;punc_: hàm dùng để loại bỏ các kí tự đặc biệt như dấu câu trong định dạng Unicode. Tham khảo: 

In [7]:
tbl = dict.fromkeys(i for i in range(sys.maxunicode)
                       if ud.category(chr(i)).startswith('P'))

def remove_punc(text)
    return text.translate(tbl)

SyntaxError: invalid syntax (<ipython-input-7-8ca530440038>, line 4)

__Ví dụ:__ Title của _real&#95;1.json_ 

In [ ]:
print(title_str)

Title của _real&#95;1.json_ sau khi loại bỏ kí tự đặc biệt: 

In [ ]:
new_string = remove_punc(title_str)
print(new_string)

### Solution

Bắt đầu xử lý các dữ liệu trong list các file JSON và lưu trong list ___preproc&#95;news___:

In [ ]:
preproc_news = pd.DataFrame(columns=['title', 'text', 'authors'])

for index, js in enumerate(json_files):
    with open(os.path.join(path_to_news, js), 'r', encoding='utf-8') as json_file:
        json_text = json.load(json_file)
        
        
        title = remove_punc(html.unescape(json_text['title']))
        text = remove_punc(html.unescape(json_text['text']))
        author = json_text['authors']
        
        preproc_news.loc[index] = [title, text, author]
        
print(preproc_news.loc[0].text)

## 2. Tokenizer & Loại StopWords

Trước hết cần biểu diễn token sau đó loại bỏ đi các stopword

Dữ liệu từ điển được lấy từ (các file được chứa trong thư mục ./corpus):
1. [deepai-solutions/core_nlp](https://github.com/deepai-solutions/core_nlp/tree/master/tokenization): 2 file _bi_\__gram.txt_ và _tri_\__gram.txt_

### 2.1. Cách 1: Sử dụng so khớp dài nhất 

Sử dụng so khớp dài nhất (longest matching) để tách các từ trong  tin tức vừa loại dấu câu

In [ ]:
import ast
import re #Regular Expression

_load_\__n_\__gram_: hàm đọc dữ liệu trong các corpus và đưa vào  chương trình với kiểu dữ liệu là ___Sets___

In [ ]:
def load_n_gram(file_path):
    with open(file_path, encoding='utf-8') as n_gram:
        words = n_gram.read()
        words = ast.literal_eval(words)
        return words

Đường dẫn đến __corpus__ và tên các file corpus.

In [ ]:
corpus_path = './corpus/'
bi_gram_path = 'bi_gram.txt'
tri_gram_path = 'tri_gram.txt'

_bi_\__gram_ và _tri_\__gram_ là các tập dữ liệu từ các corpus của chúng ta. Ở đoạn kế tiếp chúng ta có thể thấy số lượng phần tử các corpus

In [ ]:
bi_gram = load_n_gram(os.path.join(corpus_path, bi_gram_path))
tri_gram = load_n_gram(os.path.join(corpus_path, tri_gram_path))

In [ ]:
print ('2-gram: ', len(bi_gram), '& 3-gram: ', len(tri_gram))

In [ ]:
text = ud.normalize('NFC', preproc_news.loc[0].text)
word = "\w+"
token = re.findall(word, text, re.UNICODE)
print(token)

Thử nghiệm chính của _LongestMatching_. Tham khảo chủ yếu từ: [ core_nlp/tokenization/dict_models.py](https://github.com/deepai-solutions/core_nlp/blob/master/tokenization/dict_models.py)

In [ ]:
def LongestMatching(token, bi_gram, tri_gram):
    token_len = len(token)
#     print(token_len)
    cur_id = 0
    
    word_list = []
    
    done = False
    
    while (cur_id < token_len) and (not done):
        cur_word = token[cur_id]
        if(cur_id >= token_len - 1):
            word_list.append(cur_word)
            done = True
#             print(cur_id)
        else:
            next_word = token[cur_id + 1]
            bi_word = " ".join([cur_word.lower(), next_word.lower()])
#             print(bi_word)
            if(cur_id >= token_len - 2):
                    if bi_word in bi_gram:
                        word_list.append("_".join([cur_word, next_word]))
                        cur_id  = cur_id + 2
                    else: 
                        word_list.append(cur_word)
                        cur_id  = cur_id + 1
                        
            else: 
                next_next_word = token[cur_id + 2]
                tri_word = " ".join([bi_word, next_next_word.lower()])
                if tri_word in tri_gram:
                    word_list.append("_".join([cur_word, next_word, next_next_word]))
                    cur_id  = cur_id + 3
#                     print(tri_word)
                elif bi_word in bi_gram:
                    word_list.append("_".join([cur_word, next_word]))
                    cur_id = cur_id + 2
#                     print(bi_word)
                else:
                    word_list.append(cur_word)
                    cur_id = cur_id + 1
    return word_list

_text_\__tokenize_ là bộ dữ liệu sau khi đã tách các từ thành các token và có thể đưa vào mô hình train

In [ ]:
text_tokenize = LongestMatching(token, bi_gram, tri_gram)
text_for_train = " ".join(text_tokenize)
print(text_for_train)